Stats701 Homework 10  
Written By: David Li

Consulted with Inson Zeng about general approach tips

In [1]:
# Preliminary Loading of packages
import tensorflow as tf
import numpy as np
import tempfile

# Part 1: Tensorflow logo
Spent about 1 hour on this problem

## Question 1: Drawing the tensorflow logo with a constant 5 by 4 by 3 tensor

In [2]:
# As following recommendation, creating a numpy array first with 5 by 4 by 3 dimensions with 0 indexing
# 5 is the shape for height of the logo from top to bottom, 
# 4 is the shape for length in axis direction of the T projection from left to right, 
# 3 is the shape for length in axis direction of the F projection from right to left

# Initialize the dimension and lengths
logonumpy = np.zeros((5, 4, 3))

# Draw the top cross of T
logonumpy[0,:,2] = 1

# Draw the top of F
logonumpy[0,3,:] = 1

# Draw the vertical of T
logonumpy[:,2,2] = 1

# Draw the middle cube for F
logonumpy[2,2,1] = 1

# Converting to a constant tensor with shape 5 by 4 by 3
# Using the tensorflow function convert_to_tensor, which is:
#   This function converts Python objects of various types to Tensor objects. 
#   It accepts Tensor objects, numpy arrays, Python lists, and Python scalars.
logotensor = tf.convert_to_tensor(logonumpy)

# Testing
sess = tf.Session()
# Shape
print(logotensor.shape) # Expecting (5,4,3)
# Printing for verification, each instance is the top-down view of a height layer with the (0,3,2) corner as the bottom right
print(logotensor.eval(session=sess))

sess.close()



(5, 4, 3)
[[[0. 0. 1.]
  [0. 0. 1.]
  [0. 0. 1.]
  [1. 1. 1.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 1.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 1. 1.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 1.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 1.]
  [0. 0. 0.]]]


# Part 2: Building and training simple models  
Spent about 8 hours on this problem

In [4]:
# Load the data to build the logistic regression tensorflow model

# Training data
logis_xtrain = np.load("/home/david/Desktop/School/Stats701/Homework10/logistic_data/logistic_xtrain.npy")
logis_ytrain = np.load("/home/david/Desktop/School/Stats701/Homework10/logistic_data/logistic_ytrain.npy")

# Test Data
logis_xtest = np.load("/home/david/Desktop/School/Stats701/Homework10/logistic_data/logistic_xtest.npy")
logis_ytest = np.load("/home/david/Desktop/School/Stats701/Homework10/logistic_data/logistic_ytest.npy")

#### Question 1: Logistic regression with a negative log-likelihood loss

In [5]:
# As discussed in lecture, each Tensorflow model involves two steps: Building the computational graph and running it.

# First, we build the computational graph. This step involves initializing 
#  Constants, where their value never changes
#  Placeholders, where their value is obtained from elsewhere in our program (in this case, we are loading data)
#  Variables, where their value can be tuned and adjusted accordingly.

# There are no constants within the log-likelihood function

# Consider these placeholder inputs, to be initially given:
ytrue = tf.placeholder(tf.float32, [None, 1]) # utilizes a 1-column of the actual data response values
x = tf.placeholder(tf.float32, [None, 6]) # utilizes a 6-column of unspecified rows, representing the data predictors

# Consider these variable parameters, which we might change as we tune the model:
W = tf.Variable(tf.zeros([6, 1]), dtype = tf.float32) # must be 6 x 1, to correctly matrix multiply by the 6-column predictors
b = tf.Variable([0], dtype=tf.float32) # the bias parameter

# Model predictions based on given predictors
ypredicted = tf.matmul(x, W) + b # the predicted y
probpredicted = 1/(1+tf.exp(-1*ypredicted)) # the predicted probability estimate

# representing the derived negative log-likelihood loss function
logisticloss = tf.reduce_sum((ytrue * tf.log(probpredicted)) + ((1-ytrue)*tf.log(1-probpredicted)))
neglogisticloss = logisticloss * -1

#### Question 2: Estimating parameters in logistic regression

In [6]:
# To complete our model, we lastly need to estimate reasonable W and b parameters.
# This will involve gradient descent, which we approach from black-box approach from lecture notes

# First we may need parameters to use in the gradient descent optimizer. 
# Let's set the learning rate to 0.01 and the number of steps to be 500.

# Gradient descent procedure
minimizer = tf.train.GradientDescentOptimizer(0.01).minimize(neglogisticloss)

# Begin running a smaller session to estimate the W and b parameters
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# We plan on comparing training data loss before and after gradient descent optimization, and the variable values
# Before Optimization

print("W and b parameters, pre-optimization:")
print(sess.run([W, b]))
print("Training data loss, pre-optimization:")
print(sess.run(neglogisticloss, feed_dict={x: logis_xtrain, ytrue: logis_ytrain}))
print("##############################################")

# Running gradient descent procedure (the optimization)
for i in range(500):
    sess.run(minimizer, feed_dict={x: logis_xtrain, ytrue: logis_ytrain})

# After Optimization

print("W and b parameters, post-optimization:")
print(sess.run([W, b]))
print("Training data loss, post-optimization:")
print(sess.run(neglogisticloss, feed_dict={x: logis_xtrain, ytrue: logis_ytrain}))

# W is approximately [1.0921, 0.8726, 2.3463, 3.116, 5.026, 8.2909] and b is approximately [-0.21865]

W and b parameters, pre-optimization:
[array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32), array([0.], dtype=float32)]
Training data loss, pre-optimization:
1386.2922
##############################################
W and b parameters, post-optimization:
[array([[1.0921205 ],
       [0.87264544],
       [2.3463395 ],
       [3.1160364 ],
       [5.0267167 ],
       [8.290963  ]], dtype=float32), array([-0.21865678], dtype=float32)]
Training data loss, post-optimization:
722.3639


#### Question 3: Evaluating logistic regression on test data.

In [7]:
# The session still contains the optimized W and b parameters, let's keep using these

print("W and b parameters, post-optimization:")
print(sess.run([W, b]))
print("Test data loss, post-optimization:")
print(sess.run(neglogisticloss, feed_dict={x: logis_xtest, ytrue: logis_ytest}))


W and b parameters, post-optimization:
[array([[1.0921205 ],
       [0.87264544],
       [2.3463395 ],
       [3.1160364 ],
       [5.0267167 ],
       [8.290963  ]], dtype=float32), array([-0.21865678], dtype=float32)]
Test data loss, post-optimization:
180.82965


#### Question 4: Evaluating the estimated logistic parameters.

In [8]:
# Specs tell us the truth values of the W and b parameters, so let's see how off we were repesented as a squared error sum
W_truth = tf.constant([1.0, 1.0, 2.0, 3.0, 5.0, 8.0], shape=[6, 1], dtype=tf.float32) # float32 for consistency
b_truth = tf.constant([-1.0], dtype=tf.float32) # float32 for consistency
squarederror = tf.reduce_sum((W_truth - W) * (W_truth - W)) + tf.reduce_sum((b_truth - b) * (b_truth - b))

with sess.as_default():
    print("Squared error between estimated W, b  parameters and truth W, b  parameters is:", squarederror.eval())
    
sess.close()

Squared error between estimated W, b  parameters and truth W, b  parameters is: 0.85399145


-------------------------------------------------------------------------------------------------

In [10]:
# Load the data to build the normally distributed classification tensorflow model

# Training data
norm_xtrain = np.load("/home/david/Desktop/School/Stats701/Homework10/normal_data/normal_xtrain.npy")
norm_ytrain = np.load("/home/david/Desktop/School/Stats701/Homework10/normal_data/normal_ytrain.npy")

# Test Data
norm_xtest = np.load("/home/david/Desktop/School/Stats701/Homework10/normal_data/normal_xtest.npy")
norm_ytest = np.load("/home/david/Desktop/School/Stats701/Homework10/normal_data/normal_ytest.npy")

#### Question 5: Classification of normally distributed data

In [11]:
# As discussed in lecture, each Tensorflow model involves two steps: Building the computational graph and running it.

# First, we build the computational graph. This step involves initializing 
#  Constants, where their value never changes
#  Placeholders, where their value is obtained from elsewhere in our program (in this case, we are loading data)
#  Variables, where their value can be tuned and adjusted accordingly.

# There are no applicable tensor constants

# Consider these placeholder inputs, to be initially given:
ytrue = tf.placeholder(tf.float32, [None, 3]) # utilizes a 3-column of the actual data response values, in one-hot encoding style
x = tf.placeholder(tf.float32) # utilizes a 1-column of data predictors
probs = tf.placeholder(tf.float32, shape = [3])

# Consider these variable parameters, which we might change as we tune the model:
mu0 = tf.Variable(tf.zeros([1]), dtype = tf.float32)
var0 = tf.Variable(tf.ones([1]), dtype = tf.float32)
mu1 = tf.Variable(tf.zeros([1]), dtype = tf.float32)
var1 = tf.Variable(tf.ones([1]), dtype = tf.float32)
mu2 = tf.Variable(tf.zeros([1]), dtype = tf.float32)
var2 = tf.Variable(tf.ones([1]), dtype = tf.float32)

# For code readability
pie = np.pi # pi

# Establish the densities for each normally distributed class
density0 = ((1/(tf.sqrt(2*pie*var0))) * tf.exp((-1*((x-mu0)**2))/(2*var0)))
density1 = ((1/(tf.sqrt(2*pie*var1))) * tf.exp((-1*((x-mu1)**2))/(2*var1)))
density2 = ((1/(tf.sqrt(2*pie*var2))) * tf.exp((-1*((x-mu2)**2))/(2*var2)))

# Represent the cross-entropy for one observation, as defined in specs
cross_entropy=-1 * (tf.reduce_sum(ytrue*tf.log(tf.transpose([density0,density1,density2]))))

# Establish the Adagrad Optimizer, with learning rate 0.01 and number of steps as 5000
adaminimizer=tf.train.AdagradOptimizer(0.1).minimize(cross_entropy)

# Create a session for running the normal classification models
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# Tuning the means and variances
for i in range(5000):
    sess.run(adaminimizer, feed_dict = {x: norm_xtrain, ytrue: norm_ytrain})

# Estimates of class means and variances
# Class 0, mu and variance
print(sess.run([mu0, var0]))
# Class 1, mu and variance
print(sess.run([mu1, var1]))
# Class 2, mu and variance
print(sess.run([mu2, var2]))

[array([-0.9810104], dtype=float32), array([0.5114171], dtype=float32)]
[array([0.00807609], dtype=float32), array([1.0041919], dtype=float32)]
[array([3.0174267], dtype=float32), array([1.4122144], dtype=float32)]


#### Question 6: Evaluating loss on test data.

In [12]:
# Run the cross-entropy function on the test data
print(sess.run(cross_entropy, {x: norm_xtest, ytrue: norm_ytest}))

676.53253


#### Question 7: Evaluating parameter estimation on test data.

True parameters for each class:

$$\mu_0 =−1, \sigma_0^2 =0.5$$ 
$$\mu_1 = 0, \sigma_1^2 = 1$$
$$\mu_2 = 3, \sigma_2^2 = 1.5$$

In [13]:
mu0_truth = tf.constant([-1.0], dtype=tf.float32) # float32 for consistency
mu1_truth = tf.constant([0.0], dtype=tf.float32) # float32 for consistency
mu2_truth = tf.constant([3.0], dtype=tf.float32) # float32 for consistency
var0_truth = tf.constant([0.5], dtype=tf.float32) # float32 for consistency
var1_truth = tf.constant([1.0], dtype=tf.float32) # float32 for consistency
var2_truth = tf.constant([1.5], dtype=tf.float32) # float32 for consistency

squarederror = tf.reduce_sum(((mu0_truth - sess.run([mu0])[0])**2) + ((var0_truth - sess.run([var0])[0])**2) +
                             ((mu1_truth - sess.run([mu1])[0])**2) + ((var1_truth - sess.run([var1])[0])**2) +
                             ((mu2_truth - sess.run([mu2])[0])**2) + ((var2_truth - sess.run([var2])[0])**2))

with sess.as_default():
   print("Squared error between estimated mu and var parameters vs. their truth counterparts is:", squarederror.eval())

Squared error between estimated mu and var parameters vs. their truth counterparts is: 0.008583753


#### Question 8: Evaluating classification error on test data.

In [14]:
# Refer to the tensorflow normal distribution
normdists = tf.distributions.Normal(loc=[mu0,mu1,mu2], scale = [tf.sqrt(var0), tf.sqrt(var1),tf.sqrt(var2)])
# Note that I'm using variance, and the tensorflow normdistribution uses standard deviation
normdists2 = tf.distributions.Normal(loc=[mu0_truth,mu1_truth,mu2_truth], scale = [tf.sqrt(var0_truth), tf.sqrt(var1_truth),tf.sqrt(var2_truth)])

# Calculate what proportion of predictions were correct
prop_pred_correct = tf.equal(tf.argmax(tf.map_fn(lambda i: tf.log(normdists.prob(i)),x)[:,:,0],1), tf.argmax(ytrue, 1))

# Average the proportions as requested
classaccuracy = tf.reduce_mean(tf.cast(prop_pred_correct, tf.float64)) # using float64 for more precision

# Print the average classification error proportion of estimated model on test data
print(sess.run(classaccuracy, {x: norm_xtest, ytrue: norm_ytest}))



0.72


In [15]:
# For Number 4 later

# Get the probabilities of belonging to each distribution
probs = normdists2.prob([x, x, x])
# Get the index aka group of max likelihood
prediction = tf.argmax(probs)[0]
# Testing
print(sess.run(prediction, feed_dict={x: 4.0}))


2


In [ ]:
# Save the current session as a Tensorflow Model, for use in Question 4
tf.saved_model.simple_save(session = sess,
            export_dir = "/home/david/Desktop/School/Stats701/Homework10/davidjli_normal_trained/",
            inputs={"x": x},
            outputs = {"prediction":prediction})

# Close the Session
sess.close()

# Part 3: Building a Complicated Model
Spent about 2 hours on this problem

I've picked to do the Linear Model for a binary classification problem. URL: https://www.tensorflow.org/tutorials/wide

The tutorial includes instructions to download the data, but I will simply include the data in my submission for simplicity. The datafiles are adult.data and adult.test, which are included in the included lm_data folder within my submission. The command executed to download the data is below, assuming data_download.py is available for use (which the code was given in the tutorial)

In [ ]:
# %run data_download --data_dir lm_data

In [17]:
## TUTORIAL CODE ###

# Census Income Dataset metadata
CSV_COLUMN_DEFAULTS = [[0], [''], [0], [''], [0], [''], [''], [''], [''], [''],
                        [0], [0], [0], [''], ['']]
CSV_COLUMNS = [
    'age', 'workclass', 'fnlwgt', 'education', 'education_num',
    'marital_status', 'occupation', 'relationship', 'race', 'gender',
    'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
    'income_bracket'
]

# Number of rows in each set of data
num_examples = {'train': 32561, 'validation': 16281}

# Creating the Input Builder function to specify inputs for the tf.estimator model
def input_fn(data_file, num_epochs, shuffle, batch_size, buf_size):
    """
    Generate an input function for the Estimator
    Returns x, y 
    where x is a mapping of feature column names to Tensors with that feature's data
    and y is a Tensor containing labels
    """
    
    # Binary Classification, for labeling 1 if the income is over 50K and 0 if not
    def parse_csv(value):
        print('Parsing', data_file)
        columns = tf.decode_csv(value, record_defaults=CSV_COLUMN_DEFAULTS)
        features = dict(zip(CSV_COLUMNS, columns))
        labels = features.pop('income_bracket')
        return features, tf.equal(labels, '>50K')

    # using the dataset API to do extraction from the input files
    # continuous columns -> Tensor, categorical data -> SparseTensor
    dataset = tf.data.TextLineDataset(data_file)
    
    # Optionally, shuffle the dataset
    if shuffle:
        dataset = dataset.shuffle(buf_size)

    # parse_csv on the dataset
    dataset = dataset.map(parse_csv, num_parallel_calls=5)

    # Handling the epochs and shuffling
    dataset = dataset.repeat(num_epochs)
    dataset = dataset.batch(batch_size)

    iterator = dataset.make_one_shot_iterator()
    features, labels = iterator.get_next()
    return features, labels

## adjusting some feature columns to be used in the model

# defining for categorical features with specified set of values
education = tf.feature_column.categorical_column_with_vocabulary_list(
    'education', [
        'Bachelors', 'HS-grad', '11th', 'Masters', '9th', 'Some-college',
        'Assoc-acdm', 'Assoc-voc', '7th-8th', 'Doctorate', 'Prof-school',
        '5th-6th', '10th', '1st-4th', 'Preschool', '12th'])

# hashbucket for categorical features with no specified set of values
occupation = tf.feature_column.categorical_column_with_hash_bucket(
    'occupation', hash_bucket_size=1000)

# repeat for all other categorical features
marital_status = tf.feature_column.categorical_column_with_vocabulary_list(
    'marital_status', [
        'Married-civ-spouse', 'Divorced', 'Married-spouse-absent',
        'Never-married', 'Separated', 'Married-AF-spouse', 'Widowed'])

workclass = tf.feature_column.categorical_column_with_vocabulary_list(
    'workclass', [
        'Self-emp-not-inc', 'Private', 'State-gov', 'Federal-gov',
        'Local-gov', '?', 'Self-emp-inc', 'Without-pay', 'Never-worked'])

occupation = tf.feature_column.categorical_column_with_hash_bucket(
    'occupation', hash_bucket_size=1000)

# continuous features
age = tf.feature_column.numeric_column('age')
capital_gain = tf.feature_column.numeric_column('capital_gain')
hours_per_week = tf.feature_column.numeric_column('hours_per_week')

# continuous features -> categorical with bucketization
age_buckets = tf.feature_column.bucketized_column(
    age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])

# there are interactions between some predictors, these will handle the interactions
education_x_occupation = tf.feature_column.crossed_column(
    [education, 'occupation'], hash_bucket_size=1000)

age_buckets_x_education_x_occupation = tf.feature_column.crossed_column(
    [age_buckets, education, 'occupation'], hash_bucket_size=1000)

# Use Tensorflow's LinearClassifier model with the features we've collected
base_columns = [education, marital_status, occupation, age,
                capital_gain, hours_per_week, workclass]
crossed_columns = [education_x_occupation,
                   age_buckets_x_education_x_occupation]

model_dir = tempfile.mkdtemp()
model = tf.estimator.LinearClassifier(model_dir=model_dir, 
                                      feature_columns=base_columns+crossed_columns)

# parameters for the training data
train_data = "/home/david/Desktop/School/Stats701/Homework10/lm_data/adult.data"
num_epochs = 2
batch_size = 40
do_shuffle = True

# training our model
model.train(input_fn=lambda: input_fn(train_data, num_epochs, do_shuffle, 
                                      batch_size, num_examples['train']))

# parameters for the test data
test_data = "/home/david/Desktop/School/Stats701/Homework10/lm_data/adult.test"
num_epochs = 2
batch_size = 40
do_shuffle = True

# testing our model
results = model.evaluate(input_fn=lambda: input_fn(
    test_data, num_epochs, do_shuffle, batch_size, num_examples['validation']))

# print some metrics on how the model ran on the test data
for key in sorted(results):
    print('%s: %s' % (key, results[key]))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpqcd485lw', '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_task_type': 'worker', '_global_id_in_cluster': 0, '_save_summary_steps': 100, '_service': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbfe8743cc0>, '_tf_random_seed': None, '_num_worker_replicas': 1, '_task_id': 0, '_is_chief': True, '_master': ''}
Parsing /home/david/Desktop/School/Stats701/Homework10/lm_data/adult.data
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpqc

The LinearClassifier Model shows that the accuracy on the test data was around 0.8333, pretty good! I used 2 epochs for the training set and 2 epochs for the test set with batch sizes of 40 each.

# Part 4: Running Models on Google Cloud Platform
Spent about 6 hours on this problem

#### Question 1: CNN on GCP

Done all the tutorial steps

#### Question 2: Revisiting normally-distributed classifier

Done

#### Question 3: Upload normally-distributed classifier to GCP

Done

#### Question 4: Create a version

Done

#### Question 5: Create .json of single prediction

Done

#### Question 6: Do a single prediction

My prediction model with x=4 in the json file says the likely cluster is cluster 2, which is mean 3 and variance 1.5. Makes sense.